In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [ ]:
customers = ['SB', 'SD', 'MB', 'MD']

for i in customers:
  if i == 'SB':
    target_data = pd.read_pickle('/content/drive/MyDrive/Stanford-TUBerlin/CodePaper/Consumption_Matrices/{}_accounts_actual_usage'.format(i))
  else:
    df_append = pd.read_pickle('/content/drive/MyDrive/Stanford-TUBerlin/CodePaper/Consumption_Matrices/{}_accounts_actual_usage'.format(i))
    target_data = pd.concat([target_data, df_append])

In [ ]:
df_march = target_data[['2019-03-01', '2018-03-01', '2017-03-01']]
df_march.columns = [2019, 2018, 2017]


In [ ]:
df_may = target_data[['2019-05-01', '2018-05-01', '2017-05-01']]
df_may.columns = [2019, 2018, 2017]


In [ ]:
df_std = df_may.subtract(df_march).std(axis=1).to_frame()

In [ ]:
df_20 = target_data[['2020-05-01', '2020-03-01']]
df_20.columns = ['may', 'march']

In [ ]:
df_20['diff'] = df_20.may - df_20.march

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_20['change'] = np.NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:

for idx in df_std.index:
  # increase
  if df_20['diff'].loc[idx] > df_std[0].loc[idx]:
    #print(df_20['diff'].loc[idx], df_std[0].loc[idx], 1)
    df_20.loc[idx]['change'] = 1

  # decrease
  if df_20['diff'].loc[idx] < -df_std[0].loc[idx]:
    #print(df_20['diff'].loc[idx], df_std[0].loc[idx], 2)
    df_20.loc[idx]['change'] = 2
   
  # no change
  if df_20['diff'].loc[idx] < df_std[0].loc[idx] and df_20['diff'].loc[idx] > -df_std[0].loc[idx]:
    #print(df_20['diff'].loc[idx], df_std[0].loc[idx], 0)
    df_20.loc[idx]['change'] = 0


In [ ]:
increase = df_20[df_20.change == 1]


In [ ]:
np.average(increase['diff'])

136.90653405916953

In [ ]:
np.unique(df_20.change, return_counts=True)[1]/8871

array([0.36399504, 0.59046331, 0.04554165])

In [ ]:
np.unique(df_20.change, return_counts=True)

(array([0., 1., 2.]), array([3229, 5238,  404]))

In [ ]:
cust = ['SB', 'SD', 'MB', 'MD']
values = [[0,1,2,3],[0,1,2,3],[0,1,2,3],[0,1,2,3,4]]
replacement = [['high', 'medium', 'super', 'low'], ['low', 'high', 'medium', 'super'],
               ['super', 'low', 'medium', 'high'], ['low', 'medium', 'extreme','high',  'super']]

all_profiles = pd.DataFrame(index= df_20.index, columns= ['group', 'consumption', 'pattern', 'Y_covid'] )


In [ ]:
for i,_ in enumerate(cust):
  df = pd.read_pickle('/content/drive/MyDrive/Stanford-TUBerlin/CodePaper/Clustering/{}_clusters.pdf'.format(cust[i]))
  df.avg_consump.replace(values[i], replacement[i], inplace=True)
  consumption = df.avg_consump.values
  pattern = df.pattern.astype('int').values
  for k,l in enumerate(df.index):
    all_profiles.loc[l] = cust[i],consumption[k], pattern[k], np.NaN




In [ ]:
for i in all_profiles.index:
  all_profiles.Y_covid.loc[i] = df_20.change.loc[i]

In [ ]:
down = np.unique(all_profiles.group[all_profiles.Y_covid == 2], return_counts=True)
down

(array(['MB', 'MD', 'SB', 'SD'], dtype=object), array([ 63,  25, 236,  80]))

In [ ]:
up = np.unique(all_profiles.group[all_profiles.Y_covid == 1], return_counts=True)
up

(array(['MB', 'MD', 'SB', 'SD'], dtype=object),
 array([ 542,   27, 4518,  151]))

In [ ]:
no = np.unique(all_profiles.group[all_profiles.Y_covid == 0], return_counts=True)
no

(array(['MB', 'MD', 'SB', 'SD'], dtype=object),
 array([ 519,   51, 2292,  367]))

In [ ]:
all = np.unique(all_profiles.group, return_counts=True)
all

(array(['MB', 'MD', 'SB', 'SD'], dtype=object),
 array([1124,  103, 7046,  598]))

In [ ]:
down[1]/all[1]

array([0.05604982, 0.24271845, 0.03349418, 0.13377926])

In [ ]:
up[1]/all[1]

array([0.48220641, 0.26213592, 0.64121487, 0.25250836])

In [ ]:
no[1]/all[1]

array([0.46174377, 0.49514563, 0.32529095, 0.61371237])

In [ ]:
# load usage data

path_1 = '/content/drive/My Drive/Stanford-TUBerlin/Code Marie/MESA_2002_2020_cleaned.csv'

data = pd.read_csv(path_1, sep=',', index_col=0)

data['date']= pd.to_datetime(data['date'])

In [ ]:
# get Census Tracts 
all_profiles['W_LocationID'] =all_profiles.index
all_profiles.index.name = None
data = data.drop_duplicates(subset='W_LocationID')

CT = data[['W_LocationID', 'CensusTract']].merge(all_profiles, on='W_LocationID', how='right')
CT_data = [f'{x:.2f}' for x in CT.CensusTract]
CT_data = [x.replace('.', '') for x in CT_data]
rest = ['0'] *len(CT_data)
CT.CensusTract = [i+str(j) for i,j in zip(rest, CT_data)]

In [ ]:
# get Block Group
CT_final = data[['W_LocationID', 'BlockGroup']].merge(CT, on='W_LocationID', how='right')
CT_data = [f'{x:.3f}' for x in CT_final.BlockGroup]
CT_data = [x.replace('.', '') for x in CT_data]
rest = ['060590'] *len(CT_data)
CT_final.BlockGroup = [i+str(j) for i,j in zip(rest, CT_data)]

In [ ]:
CT_final.rename(columns={'CensusTract': 'TRACTCE', 'BlockGroup': 'GEOID'}, inplace= True)
CT_final

,W_LocationID,GEOID,TRACTCE,group,consumption,pattern,Y_covid
0,MW10019,060590636052,063605,SB,medium,2,1.0
1,MW10020,060590636052,063605,SB,high,3,0.0
2,MW10027,060590636052,063605,SB,high,3,1.0
3,MW10028,060590636052,063605,SB,low,3,0.0
4,MW10031,060590636052,063605,SB,medium,4,1.0
...,...,...,...,...,...,...,...
8866,MW9571,060590639071,063907,MD,low,2,1.0
8867,MW9572,060590639071,063907,MD,low,2,2.0
8868,MW9574,060590639071,063907,MD,low,2,2.0
8869,MW9576,060590639071,063907,MD,low,2,2.0


In [ ]:
# correct wrong BG numbers
wrong_BG =['060590639064', '060590638073', '060590638074', '060590639044', '060590632014', '060590636043']
new_BG = ['060590639063', '060590638072', '060590638072', '060590639043', '060590632013', '060590636042']

CT_final.GEOID.replace(wrong_BG, new_BG, inplace=True)

In [ ]:
block_feat = pd.read_pickle('/content/drive/MyDrive/Stanford-TUBerlin/Code Marie/ClusterChange_multiBaseline/GIS plots/Features_SocioDemo')

In [ ]:
block_feat.rename(columns={'Total':'totHH'}, inplace=True)

In [ ]:
merger = block_feat.merge(CT_final, on='GEOID', how='right')

In [ ]:
merger.replace(['-'], ['NaN'], inplace=True)

In [ ]:
merger.drop(columns=['GEOID', 'TRACTCE', 'W_LocationID'], inplace=True) #

In [ ]:
np.unique(merger.Y_covid, return_counts=True)

(array([0., 1., 2.]), array([3229, 5238,  404]))

In [ ]:
merger.to_pickle('/content/drive/MyDrive/Stanford-TUBerlin/CodePaper/COVID_analysis/FeatureMatrix_COVID')